In [1]:
import time
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import talib

# === CONFIG ===
SYMBOLS = [
    "XAUUSD", "EURUSD", "GBPUSD", "USDJPY", "AUDUSD",
    "USDCAD", "NZDUSD", "USDCHF", "EURJPY", "GBPJPY",
    "EURGBP", "CADJPY", "CHFJPY", "AUDJPY", "EURNZD"
]
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MIN_PROFIT_USD = 0.10
MAX_TRADES = 1
FORCE_MODE = False  # only trade on strong signals

# === INIT ===
if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None:
    raise RuntimeError(f"MT5 account_info() failed: {mt5.last_error()}")

for symbol in SYMBOLS:
    info = mt5.symbol_info(symbol)
    if info and not info.visible:
        mt5.symbol_select(symbol, True)

# === Helper: Get candle data ===
def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"Failed to get data for {symbol}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === Signal logic with 2+ bullish or bearish confirmations ===
def get_trade_signal(df, symbol):
    close = df['close'].astype(float).values
    open_ = df['open'].astype(float).values
    high = df['high'].astype(float).values
    low = df['low'].astype(float).values

    rsi = talib.RSI(close, timeperiod=14)
    if len(rsi) == 0:
        return None

    patterns = {
        "engulfing": talib.CDLENGULFING(open_, high, low, close)[-1],
        "doji": talib.CDLDOJI(open_, high, low, close)[-1],
        "morning_star": talib.CDLMORNINGSTAR(open_, high, low, close)[-1],
        "hammer": talib.CDLHAMMER(open_, high, low, close)[-1]
    }

    last_rsi = rsi[-1]
    bullish_count = sum(1 for p in patterns.values() if p > 0)
    bearish_count = sum(1 for p in patterns.values() if p < 0)

    print(f"{symbol} - RSI: {last_rsi:.2f}, Bullish: {bullish_count}, Bearish: {bearish_count}")

    if bullish_count >= 2 and last_rsi < 45:
        return "buy"
    elif bearish_count >= 2 and last_rsi > 60:
        return "sell"
    return None

# === Open trade ===
def open_trade(symbol, direction):
    tick = mt5.symbol_info_tick(symbol)
    price = tick.ask if direction == "buy" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": LOT_SIZE,
        "type": order_type,
        "price": price,
        "deviation": 20,
        "magic": 10001,
        "comment": "safe_trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"✅ OPENED {direction.upper()} on {symbol} at {price}")
    else:
        print(f"❌ FAILED to open {symbol}: {result.retcode}")
        print(result._asdict())

# === Check & close trade if profit is reached ===
def check_and_close():
    for symbol in SYMBOLS:
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            pos = positions[0]
            profit = pos.profit
            print(f"{symbol} live profit: ${profit:.2f}")
            if profit >= MIN_PROFIT_USD:
                close_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask
                close_req = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": pos.volume,
                    "type": close_type,
                    "position": pos.ticket,
                    "price": price,
                    "deviation": 20,
                    "magic": 10001,
                    "comment": "take_profit",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }
                result = mt5.order_send(close_req)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ CLOSED {symbol} with profit ${profit:.2f}")
                else:
                    print(f"❌ FAILED to close {symbol}: {result.retcode}")

# === MAIN LOOP ===
print("🚀 Running high-confidence bot (capital protected)...")
try:
    while True:
        check_and_close()
        time.sleep(2)  # short delay for profit check

        if len(mt5.positions_get() or []) >= MAX_TRADES:
            print("⛔ Max trade active. Waiting...")
            time.sleep(10)
            continue

        for symbol in SYMBOLS:
            if not any(pos.symbol == symbol for pos in mt5.positions_get() or []):
                df = get_data(symbol, TIMEFRAME, 100)
                signal = get_trade_signal(df, symbol)
                if signal:
                    open_trade(symbol, signal)
                time.sleep(2)  # small delay between symbols

except KeyboardInterrupt:
    print("🛑 Bot manually stopped.")
finally:
    mt5.shutdown()



🚀 Running high-confidence bot (capital protected)...
XAUUSD - RSI: 33.06, Bullish: 1, Bearish: 0
EURUSD - RSI: 46.53, Bullish: 1, Bearish: 0
GBPUSD - RSI: 52.85, Bullish: 1, Bearish: 0
USDJPY - RSI: 40.34, Bullish: 1, Bearish: 0
AUDUSD - RSI: 55.14, Bullish: 0, Bearish: 0
USDCAD - RSI: 49.65, Bullish: 0, Bearish: 0
NZDUSD - RSI: 46.42, Bullish: 0, Bearish: 0
USDCHF - RSI: 50.93, Bullish: 1, Bearish: 0
EURJPY - RSI: 35.20, Bullish: 0, Bearish: 0
GBPJPY - RSI: 37.53, Bullish: 0, Bearish: 0
EURGBP - RSI: 44.86, Bullish: 1, Bearish: 0
CADJPY - RSI: 40.27, Bullish: 0, Bearish: 0
CHFJPY - RSI: 36.87, Bullish: 0, Bearish: 0
AUDJPY - RSI: 41.68, Bullish: 0, Bearish: 0
EURNZD - RSI: 54.81, Bullish: 0, Bearish: 0
XAUUSD - RSI: 31.07, Bullish: 0, Bearish: 0
EURUSD - RSI: 45.71, Bullish: 1, Bearish: 0
GBPUSD - RSI: 50.35, Bullish: 0, Bearish: 0
USDJPY - RSI: 42.42, Bullish: 0, Bearish: 0
AUDUSD - RSI: 60.28, Bullish: 0, Bearish: 0
USDCAD - RSI: 52.34, Bullish: 0, Bearish: 0
NZDUSD - RSI: 54.73, Bu

RuntimeError: Failed to get data for USDJPY